In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from vnstock import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import yfinance as yf
import os
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
data_ticker = pd.read_excel("C:\\Users\\Chi Bao\\OneDrive\\Desktop\\Đồ án tốt nghiệp\\data_ticker.xlsx")

FI = ['GC=F','SI=F','CL=F','^GSPC','^DJI','^IXIC','^RUT','^FTSE','^N225','^NYA']
df = pd.DataFrame()
for symbol in FI:
    data = yf.download(symbol, start='2012-03-20', end='2023-09-06')
    df[symbol] = data['Adj Close']

column_mapping = {
    'GC=F': 'Gold',
    'SI=F': 'Silver',
    'CL=F': 'Crude Oil',
    '^GSPC': 'S&P 500',
    '^DJI': 'Dow Jones',
    '^IXIC': 'NASDAQ',
    '^RUT': 'Russell 2000',
    '^FTSE': 'FTSE 100',
    '^N225': 'Nikkei 225',
    '^NYA': 'NYSE'
}

df = df.rename(columns=column_mapping)
df.dropna()

returns_df = df.pct_change()*100
returns_df.columns = [col + '_return' for col in returns_df.columns]
returns_df = returns_df.dropna()

###

FI = ['VNINDEX','HNX','VN30','HNX30','UPCOM']
data = stock_historical_data('VNINDEX', '2012-03-19', '2023-09-05', "1D", "index")
d_new = data.set_index('time')
d_new['VNINDEX'] = data['close']
df1 = pd.DataFrame()
for symbol in FI:
    data = stock_historical_data(symbol, '2012-03-19', '2023-09-05', "1D", "index")
    data = data.set_index('time')
    df1[symbol] = data['close']
df1 = df1.dropna()

returns_df1 = df1.pct_change()*100
returns_df1.columns = [col + '_return' for col in returns_df1.columns]
returns_df1 = returns_df1.dropna()
returns_df1.head(42069)

###

lagreturn = pd.DataFrame()

for column in returns_df.columns:
    lagreturn[f'{column}_Lag1'] = returns_df[column].shift(1)
for column in returns_df.columns:
    lagreturn[f'{column}_Lag2'] = returns_df[column].shift(2)
lagreturn = lagreturn.dropna()

lagreturn1 = pd.DataFrame()
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag1'] = returns_df1[column].shift(1)
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag2'] = returns_df1[column].shift(2)
lagreturn1 = lagreturn1.dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
def do_tre_vnstock(symbol, start_date, end_date, lags=10): 
    ts = stock_historical_data(symbol,start_date, end_date)
    ts = ts.dropna(subset=['close'])


    tsret = pd.DataFrame(index=ts.index)
    tsret['time']=ts['time']
    tsret["Today"] = ts["close"].pct_change()*100

    for i,x in enumerate(tsret["Today"]):
        if (abs(x) < 0.0001):
            tsret["Today"][i] = 0.0001

    for i in range(0, lags):
        tsret["Lag%s" % str(i+1)] = tsret["Today"].shift(i+1)

    tsret["Direction"] = np.sign(tsret["Today"])
    tsret = tsret.dropna()
    tsret = tsret.set_index('time')
    bpt = pd.merge(tsret, lagreturn, left_index=True, right_index=True)
    bpt1 = pd.merge(bpt, lagreturn1, left_index=True, right_index=True)
    bpt1.fillna(0, inplace=True)
    bpt1 = bpt1[(bpt1 != 0).all(axis=1)]
    return bpt1

In [6]:
file_path = r"C:\Users\Chi Bao\OneDrive\Desktop\Đồ án tốt nghiệp\Code\RF\data_ticker - Copy.xlsx"
result_file_path = r"C:\Users\Chi Bao\OneDrive\Desktop\Đồ án tốt nghiệp\Code\RF\Kết quả chưa cải thiện.xlsx"

In [7]:
#MODEL
list_i = []
list_return_price = []
list_rmse, list_mse, list_mae = [], [], []
list_r2_train = []
list_r2_test = []

original_data = pd.read_excel(file_path)

if os.path.exists(result_file_path):
    result_df = pd.read_excel(result_file_path)
else:
    result_df = pd.DataFrame(columns=['Stock', 'Predict', 'R2_train', 'R2_test', 'RMSE', 'MSE', 'MAE'])
    
try:     
    for i in original_data['ticker']:
        try:
            data = do_tre_vnstock(i, '2012-03-19', '2023-09-05', lags=10)

            X = data.loc[:, ~data.columns.isin(['Today'])]
            y = data["Today"]

            model = RandomForestRegressor(n_estimators=20, random_state=12)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12, shuffle=False)
            model.fit(X_train, y_train)
            yhat_test = model.predict(X_test)
            r2_train = model.score(X_train, y_train)
            r2_test = model.score(X_test, y_test)
            rmse = mean_squared_error(y_test, yhat_test, squared=False)
            mse = mean_squared_error(y_test, yhat_test, squared=True)
            mae = mean_absolute_error(y_test, yhat_test)

            X_predict = [X.iloc[-1].values.tolist()]
            Y_predict = model.predict(X_predict)

            list_i.append(i)
            list_return_price.append(Y_predict)
            list_r2_train.append(r2_train)
            list_r2_test.append(r2_test)
            list_rmse.append(rmse)
            list_mse.append(mse)
            list_mae.append(mae)

            print('Stock:',i,f' Predict: {Y_predict.item():.2f}',f' R2_train: {r2_train:.2f}',f' R2_test: {r2_test:.2f}',
                f' RMSE: {rmse:.2f}', f' MSE: {mse:.2f}', f' MAE: {mae:.2f}')
            
            result_df = pd.concat([result_df, pd.DataFrame({
                'Stock': [i],
                'Predict': [Y_predict.item()],
                'R2_train': [r2_train],
                'R2_test': [r2_test],
                'RMSE': [rmse],
                'MSE': [mse],
                'MAE': [mae]
            })], ignore_index=True)

            original_data = original_data[original_data['ticker'] != i]
            result_df.to_excel(result_file_path, index=False)
            
        except Exception as e:
            print(f"Stock: {i} -Unpredictable!\n" f'{e}')
            result_df.to_excel(result_file_path, index=False)
            original_data.to_excel(file_path, index=False)
            continue

except KeyboardInterrupt:
    print("Dừng vòng for. Kết quả lưu trong file:", result_file_path)
    result_df.to_excel(result_file_path, index=False)
    original_data.to_excel(file_path, index=False)

print("Đang tạm dừng!")

Stock: SSI  Predict: -1.35  R2_train: 0.93  R2_test: 0.53  RMSE: 1.48  MSE: 2.20  MAE: 1.20
Stock: BCM  Predict: -0.69  R2_train: 0.91  R2_test: 0.32  RMSE: 0.90  MSE: 0.80  MAE: 0.63
Stock: VHM  Predict: 1.23  R2_train: 0.92  R2_test: 0.48  RMSE: 1.61  MSE: 2.58  MAE: 1.16
Stock: VIC  Predict: 0.87  R2_train: 0.91  R2_test: 0.35  RMSE: 1.65  MSE: 2.72  MAE: 1.07
Stock: VRE  Predict: -1.26  R2_train: 0.92  R2_test: 0.54  RMSE: 1.22  MSE: 1.49  MAE: 0.89
Stock: BVH  Predict: 1.64  R2_train: 0.92  R2_test: 0.39  RMSE: 0.92  MSE: 0.85  MAE: 0.72
Stock: POW  Predict: 0.85  R2_train: 0.93  R2_test: 0.50  RMSE: 1.17  MSE: 1.37  MAE: 0.90
Stock: GAS  Predict: 0.86  R2_train: 0.92  R2_test: 0.43  RMSE: 0.81  MSE: 0.65  MAE: 0.67
Stock: ACB  Predict: 0.69  R2_train: 0.91  R2_test: 0.47  RMSE: 0.93  MSE: 0.87  MAE: 0.71
Stock: BID  Predict: 0.90  R2_train: 0.92  R2_test: 0.54  RMSE: 1.13  MSE: 1.28  MAE: 0.86
Stock: CTG  Predict: 1.31  R2_train: 0.92  R2_test: 0.39  RMSE: 1.23  MSE: 1.51  MAE: 0